<a href="https://colab.research.google.com/github/serrve1/githubtest/blob/main/%EA%B5%B0%EC%A7%91%ED%99%94%EB%A5%BC%EC%9D%B4%EC%9A%A9%ED%95%9C%EA%B2%BD%EA%B8%B0%EB%B6%84%EC%84%9D(project1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# # 한글폰트

경기분석을 군집화로 해보고 싶은데, 일단 k-means를 한번 생각해볼 수 있음 알고리즘을 제대로 이해하고 있는게 아니기 때문에 위험할 수 있으나 일단....해보자

# 1. 기본 패키지 설치

1. 기본패키지설치
2. 데이터 불러오기
2-1. 데이터프레임으로 저장
2-2. 자료구조살피기
info, columns
3. 데이터 전처리
drop => 일단 궁금하지 않은거제거
 3-1 EDA
  - 데이터프레임. describe().round(3)
  - sns(시본라이브러리).pairplot(df)
 3-2 데이터 표준화
  - standardscalar()함수 이용

In [ ]:
# 기본패키지 설치
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# 사이킷런
from sklearn.preprocessing import StandardScaler# 스케일링이 필요
from sklearn.cluster import KMeans # 케이민즈 알고리즘

### 2. 데이터불러오기

2000년 ~ 2023년 12월까지 가져오기
고용 : 실업률, 실업수당 청구건수
금리 : 미국 기준금리, 신용스프레드, 장단기 스프레드
물가 : PCE, 통화량(M2)
실물경제(제조업+주택) : 소매판매, 설비투자지수, 주택가격지수
# 분기로 묶을 거임 => 분기를 군집화 할 예정
# 거리기반 = 거리 조정이 필요, 시계열 자료의 정상성은 ?
=> 필요한지 일단 모르겠고 그대로  

#pmi제조업 지수는 시클리컬 하기 때문에 매우 중요
제조업 지수를 구할 수 없었고 신규주문으로 대체해보기

In [ ]:
# 고용 관련 지표를 가져오기
# 실업률, 신규실업수당 청구건수 2가지
#!pip install full-fred

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
BAMLH0A0HYM2 = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/BAMLH0A0HYM2(신용스프레드).csv')
FEDFUNDS = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/FEDFUNDS(기준금리)).csv')
ICSA =pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/ICSA(신규실업수당청구).csv')
M2= pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/M2.csv')
PCE = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/PCE.csv')
PNFI = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/PNFI(설비투자지수).csv')
RSXFS = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/RSXFS(소매판매).csv')
T10Y2Y = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/T10Y2Y(장단기금리차).csv')
UNRATE = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/UNRATE.csv')
USSTHPI = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/USSTHPI(주택가격지수).csv')
UMTMNO = pd.read_csv('/content/drive/MyDrive/프로젝트1/dataset/UMTMNO(제조업신규주문).csv')

 전체 지표들을 시각화할 것임 1. 추세를 제거하고 변동을 봐야할 것 같음 2. 그렇게 하려면 차분을 통해서 제거해야하나? 아님 다른 방법으로 제거해야하나? 3. 정상성이 중요하다면 ADF 검증이 필요한가? 4. Kmeans를 그냥 써보고 싶은데 정규화하면되나
 5. 2000~2022년까지 학습해보기 (코로나 포함)
 6. 분기별로 묶어서 구간 보기


In [ ]:
# 2번돌리면 에러 사라짐
# object를 숫자로 처리하는 한가지 방법
BAMLH0A0HYM2.dropna(inplace = True)
BAMLH0A0HYM2.BAMLH0A0HYM2 = BAMLH0A0HYM2.BAMLH0A0HYM2.replace('.', pd.NaT)
BAMLH0A0HYM2.BAMLH0A0HYM2 = pd.to_numeric(BAMLH0A0HYM2.BAMLH0A0HYM2)
#BAMLH0A0HYM2.BAMLH0A0HYM2

# object => 정수로 변경
T10Y2Y.T10Y2Y = T10Y2Y.T10Y2Y.replace('.', pd.NaT)
T10Y2Y.T10Y2Y = pd.to_numeric(T10Y2Y.T10Y2Y,errors='coerce')

In [ ]:
# 결측치는 전부 제거해주기
BAMLH0A0HYM2.dropna(inplace = True)
FEDFUNDS.dropna(inplace = True)
ICSA.dropna(inplace = True)
M2.dropna(inplace = True)
PCE.dropna(inplace = True)
PNFI.dropna(inplace = True)
RSXFS.dropna(inplace = True)
T10Y2Y.dropna(inplace = True)
UNRATE.dropna(inplace = True)
USSTHPI.dropna(inplace = True)
UMTMNO.dropna(inplace = True)

In [ ]:
BAMLH0A0HYM2.info() #타입이 실수형이어야

In [ ]:
list = [BAMLH0A0HYM2, FEDFUNDS, ICSA, M2, PCE,PNFI,RSXFS,T10Y2Y,UNRATE,USSTHPI,UMTMNO ]
#BAMLH0A0HYM2['DATE'] = pd.to_datetime(BAMLH0A0HYM2['DATE'])
# 모든 Date를 datetime으로 바꿔줌
for i in list:
    i['DATE'] = pd.to_datetime(i['DATE'])

In [ ]:
# # 'Year'와 'Month' 컬럼 추가
#BAMLH0A0HYM2_ = pd.DataFrame() # 분기별로 묶은 새로운 데이터 프레임 형성
# # 'Year'와 'Month'를 기준으로 그룹화하여 각 그룹에 대해 평균 계산, 예를들어 분기
BAMLH0A0HYM2['Year'] = BAMLH0A0HYM2['DATE'].dt.year
BAMLH0A0HYM2['Month'] = BAMLH0A0HYM2['DATE'].dt.month
BAMLH0A0HYM2= BAMLH0A0HYM2.groupby(['Year', 'Month']).mean().reset_index()
#BAMLH0A0HYM2 이 데이터가 가장 많이 존재하므로 이걸 기준으로 합칠 것
#list = [BAMLH0A0HYM2, FEDFUNDS, ICSA, M2, PCE,PNFI,RSXFS,T10Y2Y,UNRATE,USSTHPI,UMTMNO ]
BAMLH0A0HYM2['BAMLH0A0HYM2_분기평균'] = BAMLH0A0HYM2.iloc[:,2].rolling(window = 3).mean()

In [ ]:
BAMLH0A0HYM2.loc[ 0,'BAMLH0A0HYM2_분기평균'] = BAMLH0A0HYM2.iloc[0]['BAMLH0A0HYM2'] # 2000년 1월은 채워넣어줘야함
BAMLH0A0HYM2.iloc[1,-1] =  4.902381
# 결측치가 있는 행만 뽑고 싶을 때
BAMLH0A0HYM2[BAMLH0A0HYM2.isna().any(axis=1)]

In [ ]:
# 2번째 데이터처리, 월별로 존재
FEDFUNDS['Year'] = FEDFUNDS['DATE'].dt.year
FEDFUNDS['Month'] = FEDFUNDS['DATE'].dt.month
FEDFUNDS= FEDFUNDS.groupby(['Year', 'Month']).mean().reset_index()

FEDFUNDS['FEDFUNDS_분기평균'] = FEDFUNDS.iloc[:,2].rolling(window = 3).mean()
FEDFUNDS

In [ ]:
FEDFUNDS[FEDFUNDS.isna().any(axis=1)]
# 결측치 처리
FEDFUNDS.iloc[0,-1] =  5.45
FEDFUNDS.iloc[1,-1] =  5.73
FEDFUNDS[FEDFUNDS.isna().any(axis=1)]

In [ ]:
# 3번째 데이터 처리 ICSA , 월별로 존재 대신 2023년부터 없음
ICSA['Year'] = ICSA['DATE'].dt.year
ICSA['Month'] = ICSA['DATE'].dt.month
ICSA= ICSA.groupby(['Year', 'Month']).mean().reset_index()
ICSA['ICSA_분기평균'] = ICSA.iloc[:,2].rolling(window = 3).mean()
ICSA

In [ ]:
ICSA[ICSA.isna().any(axis=1)]
ICSA.iloc[0,-1] =  288400.0
ICSA[ICSA.isna().any(axis=1)]

In [ ]:
# 4번째 데이터 m2
M2['Year'] = M2['DATE'].dt.year
M2['Month'] = M2['DATE'].dt.month
M2= M2.groupby(['Year', 'Month']).mean().reset_index()
M2['M2_분기평균'] = M2.iloc[:,2].rolling(window = 3).mean()
M2

In [ ]:
M2['M2_분기평균'].iloc[0] = 4666.2
M2[M2.isna().any(axis=1)]

In [ ]:
# 5번째 데이터 PCE # 1,4,7,10 이런식으로 있다는게 문제
PCE['Year'] = PCE['DATE'].dt.year
PCE['Month'] = PCE['DATE'].dt.month
PCE= PCE.groupby(['Year', 'Month']).mean().reset_index()

PCE['PCE_분기평균'] = PCE.iloc[:,2].rolling(window = 3).mean()
PCE

In [ ]:
PCE[PCE.isna().any(axis=1)]
PCE.iloc[0,-1] =  2.1
PCE.iloc[1,-1] =  1.4
PCE[PCE.isna().any(axis=1)]

In [ ]:
# 6번째 데이터 PNFI
PNFI['Year'] = PNFI['DATE'].dt.year
PNFI['Month'] = PNFI['DATE'].dt.month
PNFI= PNFI.groupby(['Year', 'Month']).mean().reset_index()

PNFI['PNFI_분기평균'] = PNFI.PNFI # 나중에 전처리하고 다시 돌아오니 이거 만들 필요가 없었음

In [ ]:
PNFI[PNFI.isna().any(axis=1)]
PNFI.iloc[0,-1] =  1450.250
PNFI.iloc[1,-1] =  1498.729
PNFI[PNFI.isna().any(axis=1)]

In [ ]:
# 7번째 데이터 RSXFS
RSXFS['Year'] = RSXFS['DATE'].dt.year
RSXFS['Month'] = RSXFS['DATE'].dt.month
RSXFS= RSXFS.groupby(['Year', 'Month']).mean().reset_index()

RSXFS['RSXFS_분기평균'] = RSXFS.iloc[:,2].rolling(window = 3).mean()
RSXFS

In [ ]:
RSXFS[RSXFS.isna().any(axis=1)]
RSXFS.iloc[0,-1] =  243436.0
RSXFS.iloc[1,-1] =  247133.0
RSXFS[RSXFS.isna().any(axis=1)]

In [ ]:
# 8번째 데이터 T10Y2Y
T10Y2Y['Year'] = T10Y2Y['DATE'].dt.year
T10Y2Y['Month'] = T10Y2Y['DATE'].dt.month
T10Y2Y= T10Y2Y.groupby(['Year', 'Month']).mean().reset_index()

T10Y2Y['T10Y2Y_분기평균'] = T10Y2Y.iloc[:,2].rolling(window = 3).mean()
T10Y2Y

In [ ]:
T10Y2Y[T10Y2Y.isna().any(axis=1)]
T10Y2Y.iloc[0,-1] =  0.221
T10Y2Y.iloc[1,-1] =  -0.091
T10Y2Y[T10Y2Y.isna().any(axis=1)]

In [ ]:
# 9번째 데이터 UNRATE
UNRATE['Year'] = UNRATE['DATE'].dt.year
UNRATE['Month'] = UNRATE['DATE'].dt.month
UNRATE= UNRATE.groupby(['Year', 'Month']).mean().reset_index()

UNRATE['UNRATE_분기평균'] = UNRATE.iloc[:,2].rolling(window = 3).mean()
UNRATE

In [ ]:
UNRATE[UNRATE.isna().any(axis=1)]
UNRATE.UNRATE_분기평균.iloc[0] = 4.0

In [ ]:
# 10번째 데이터 USSTHPI
USSTHPI['Year'] = USSTHPI['DATE'].dt.year
USSTHPI['Month'] = USSTHPI['DATE'].dt.month
USSTHPI= USSTHPI.groupby(['Year', 'Month']).mean().reset_index()

USSTHPI['USSTHPI_분기평균'] = USSTHPI.iloc[:,2].rolling(window = 3).mean()
USSTHPI

In [ ]:
USSTHPI[USSTHPI.isna().any(axis=1)]
USSTHPI.iloc[0,-1] =  230.15
USSTHPI.iloc[1,-1] =  233.89
USSTHPI[USSTHPI.isna().any(axis=1)]

In [ ]:
# 11번째 데이터 UMTMNO
UMTMNO['Year'] = UMTMNO['DATE'].dt.year
UMTMNO['Month'] = UMTMNO['DATE'].dt.month
UMTMNO= UMTMNO.groupby(['Year', 'Month']).mean().reset_index()

UMTMNO['UMTMNO_분기평균'] = UMTMNO.iloc[:,2].rolling(window = 3).mean()
UMTMNO

In [ ]:
UMTMNO[UMTMNO.isna().any(axis=1)]
UMTMNO.iloc[0,-1] =  337504.0
UMTMNO.iloc[1,-1] =  370003.0
UMTMNO[UMTMNO.isna().any(axis=1)]

In [ ]:
# 리스트 재지정
list = [BAMLH0A0HYM2, FEDFUNDS, ICSA, M2, PCE,PNFI,RSXFS,T10Y2Y,UNRATE,USSTHPI,UMTMNO ]
df_ = BAMLH0A0HYM2[['Year','Month']] # 데이터 개수가 가장 많으니까
# 모든 데이터를 데이터프레임화
for i in list :
    df_ =  pd.merge(df_, i, on= ['Year', 'Month'], how='left')
    df_ = df_.loc[df_.Year != 2024]

In [ ]:
# 결측치 확인과 처리 # 2000년 ~ 2021년까지 학습할 것 같음 2023년 데이터가 통으로 없는게 있어서
df_ = df_.loc[df_.Year <= 2021]

In [ ]:
# 합치면서 사라진 결측치 처리하고 분기별로 뽑기
df_ = df_.loc[df_['Month'].isin([1, 4, 7, 10])]
df_

In [ ]:
df_.columns

In [ ]:
df_ = df_[['Year','Month','BAMLH0A0HYM2_분기평균', 'FEDFUNDS_분기평균', 'ICSA_분기평균', 'M2_분기평균', 'PCE_분기평균', 'PNFI_분기평균', 'RSXFS_분기평균','T10Y2Y_분기평균', 'UNRATE_분기평균','USSTHPI_분기평균','UMTMNO_분기평균']]

In [ ]:
#df_.fillna(metode = 'bfill')
df_.M2_분기평균 = df_.M2_분기평균.fillna(method = 'bfill')
df_.UNRATE_분기평균 = df_.UNRATE_분기평균.fillna(method = 'bfill')
df_.UMTMNO_분기평균 = df_.UMTMNO_분기평균.fillna(method = 'bfill')
df_

In [ ]:

# 한글 폰트 사용을 위해서 세팅
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
ax = df_.plot(x='Year', y= ['BAMLH0A0HYM2_분기평균',
       'FEDFUNDS_분기평균', 'ICSA_분기평균', 'M2_분기평균',
        'PCE_분기평균', 'PNFI_분기평균',
       'RSXFS_분기평균','T10Y2Y_분기평균', 'UNRATE_분기평균',
       'USSTHPI_분기평균','UMTMNO_분기평균'], kind='line')

# 그래프 출력
ax.set_yscale('linear')  # 00.set_yscale() 함수log를 입력한다면 로그 스케일로 변경
plt.title('경제데이터')
plt.figure(figsize =(20,10))
plt.show()

In [ ]:
# 스케일이 작은 것끼리 모아놓음
# 한글 폰트 사용을 위해서 세팅
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
ax = df_.plot(x='Year', y= ['BAMLH0A0HYM2_분기평균',
       'FEDFUNDS_분기평균',
        'PCE_분기평균',
       'T10Y2Y_분기평균', 'UNRATE_분기평균',
       ], kind='line')

# 그래프 출력
ax.set_yscale('linear')  # 00.set_yscale() 함수에 log를 입력한다면 로그 스케일로 변경
plt.title('경제데이터')
plt.figure(figsize =(20,2))
plt.show()

In [ ]:
import numpy as np

# 그래프 크기 조정
가로, 세로 = 25, 12  # 원하는 크기로 조절
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(가로, 세로))  # 3행 4열의 서브 플롯 생성

# 각 컬럼에 대한 서브 플롯 그리기
for i, col in enumerate(['BAMLH0A0HYM2_분기평균',
       'FEDFUNDS_분기평균', 'ICSA_분기평균', 'M2_분기평균', 'PCE_분기평균', 'PNFI_분기평균',
       'RSXFS_분기평균','T10Y2Y_분기평균', 'UNRATE_분기평균',
       'USSTHPI_분기평균','UMTMNO_분기평균']):
    df_.plot(x='Year', y=col, kind='line', ax=axs[i//4, i%4], color='blue')
    axs[i//4, i%4].set_title(f'{col} 서브 플롯')

# 그래프 출력
plt.show()

In [ ]:
#M2, PNFL, RSXFS, USSTHPI,UMTMNO 추세를 조정 (1차 차분)

df_['M2_분기평균'] = df_['M2_분기평균'].diff().fillna(method = 'bfill')
df_['PNFI_분기평균'] = df_['PNFI_분기평균'].diff().fillna(method = 'bfill')
df_['RSXFS_분기평균'] = df_['RSXFS_분기평균'].diff().fillna(method = 'bfill')
df_['USSTHPI_분기평균'] = df_['USSTHPI_분기평균'].diff().fillna(method = 'bfill')
df_['UMTMNO_분기평균'] = df_['UMTMNO_분기평균'].diff().fillna(method = 'bfill')

# # 차분 후 그래프 다시 확인
# # 그래프 크기 조정
# 가로, 세로 = 25, 12  # 원하는 크기로 조절
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(가로, 세로))  # 3행 4열의 서브 플롯 생성

# # 각 컬럼에 대한 서브 플롯 그리기
for i, col in enumerate(['BAMLH0A0HYM2_분기평균',
        'FEDFUNDS_분기평균', 'ICSA_분기평균', 'M2_분기평균', 'PCE_분기평균', 'PNFI_분기평균',
        'RSXFS_분기평균','T10Y2Y_분기평균', 'UNRATE_분기평균',
        'USSTHPI_분기평균','UMTMNO_분기평균']):
     df_.plot(y=col, kind='line', ax=axs[i//4, i%4], color='blue')
     axs[i//4, i%4].set_title(f'{col} 서브 플롯')

# # 그래프 출력
plt.show()



https://www.kaggle.com/code/tanmay111999/clustering-pca-k-means-dbscan-hierarchical

* 이제 군집화

In [ ]:
# 3개의 데이터는 거의다 결측치가 있는데  합치는과정에서인지 아니면 처음부터 없었는지 파악

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

df_['Year'] = df_['Year'].astype(str)
df_['Month'] = df_['Month'].astype(str)
df_['DATE'] = df_['Year'] +'-'+ df_['Month']
df_ = df_.drop(columns = ['Year', 'Month']) # 연,월 컬럼 제거
df_['DATE'] = pd.to_datetime(df_['DATE'])
df_.set_index('DATE',inplace = True) # 데이트를 인덱스로
# 데이트를 datetime으로 변경해라

In [ ]:
df_ # date의 일(day)는 제거해야함

https://hweejin.tistory.com/entry/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81-%EB%B6%84%EC%84%9D-3-%EC%8A%A4%EC%BC%80%EC%9D%BC-%EC%A1%B0%EC%A0%95
스케일러설명

In [ ]:
# 피쳐 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_)
scaled_df = scaler.transform(df_)
scaled_df = pd.DataFrame(index=df_.index ,data= scaled_df, columns=df_.columns)
scaled_df

In [ ]:


# 차원축소  - 적절한 차원은 얼마인가?
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(scaled_df)
scaled_df['pca_x'] = pca_transformed[:, 0]
scaled_df['pca_y'] = pca_transformed[:, 1]
scaled_df.head()


kmeans = KMeans(n_clusters =4, init = 'k-means++', max_iter= 300, random_state= 0) # 군집을 4개로
kmeans.fit(scaled_df)


In [ ]:
# 그래프 크기 조정
가로, 세로 = 25, 12  # 원하는 크기로 조절
fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(가로, 세로))  # 4행 5열의 서브 플롯 생성

# 각 컬럼에 대한 서브 플롯 그리기

for i, col in enumerate(scaled_df.columns):
    scaled_df.plot( y=col, kind='line', ax=axs[i//4, i%4], color='blue')
    axs[i//4, i%4].set_title(f'{col} 서브 플롯')

# 그래프 출력
plt.show()




In [ ]:
# KMeans 군집화와 PCA 변환 결과를 산점도로 표현합니다.
plt.scatter(scaled_df['pca_x'], scaled_df['pca_y'], c=kmeans.labels_, cmap='Blues', marker='o', alpha=0.8, edgecolors='w')

# 각 클러스터의 중심점을 빨간색 'X'로 표시합니다.
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=200, c='red', marker='X', label='중심점')

plt.title('KMeans 군집화와 PCA 변환 결과')
plt.xlabel('PCA 주성분 1')
plt.ylabel('PCA 주성분 2')
# plt.legend()
plt.show()

# 스케일링 안하면 이렇게 큰 값의 영향을 많이 받음

In [ ]:
# 1. 연도조정 , 2. PCA 컴포넌트 크기 ,PCA 여부 3. K-MEANS K개수, 그리고 K-MEANS 하는게 맞는지
# 기초통계량
scaled_df.describe()